In [2]:
import imp
import os
import cv2
import pdb
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool
import torch
from torch import optim
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
# from torch.nn.functional import InterpolationMode
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import logging 
import datetime
import sys
import cv2
import matplotlib.pyplot as plt
import PIL

In [3]:
gloss_df_path = "data_validation/processed_gloss.csv"
gloss_df = pd.read_csv(gloss_df_path)
gloss_df.dropna(inplace=True)
gloss_df.replace(to_replace="ASHAG", value="AŞAĞI", inplace=True)
gloss_df['glossRange'] = gloss_df['glossEnd'] - gloss_df['glossStart']
# gloss_df.head()

In [4]:
classes = sorted(gloss_df.gloss.unique())
num_classes = len(classes)

In [5]:
class_to_idx = {classes[i]: i for i in range(len(classes))}
idx_to_class = {i: classes[i] for i in range(len(classes))}

In [6]:
import torchviz
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler
from torchvision.models import squeezenet1_1, SqueezeNet1_1_Weights
from torchvision import transforms

In [7]:
from pytorchvideo.data import labeled_video_dataset

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute,   
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomAdjustSharpness,
    Resize,
    RandomHorizontalFlip
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [8]:
video_transform = Compose([
    ApplyTransformToKey(key="video",
    transform=Compose([
        UniformTemporalSubsample(25),
        Lambda(lambda x: x/255),
        Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
        # RandomShortSideScale(min_size=256, max_size=512),
        CenterCropVideo(256),
        RandomHorizontalFlip(p=0.5),
    ]),
    ),
])

In [9]:
train_path = "/home/toghrul/SLR/data/binary-data/train"
val_path = "/home/toghrul/SLR/data/binary-data/val"
test_path = "/home/toghrul/SLR/data/binary-data/test"

In [10]:
bin_classes = []

for gloss in os.listdir(test_path):
    bin_classes.append(gloss)

In [11]:
bin_classes = sorted(bin_classes)
bin_class_to_idx = {bin_classes[i]: i for i in range(len(bin_classes))}
bin_idx_to_class = {i: bin_classes[i] for i in range(len(bin_classes))}
bin_idx_to_class

{0: 'MƏN', 1: 'VAR'}

In [12]:
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts
from sklearn.metrics import classification_report
import torchmetrics

In [13]:
# video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)

In [14]:
# dataset = labeled_video_dataset(train_path, transform=video_transform, clip_sampler=make_clip_sampler('random', 2), decode_audio=False)

# loader = DataLoader(dataset, batch_size=8, num_workers=0, pin_memory=True)

In [15]:
# batch = next(iter(loader))

In [16]:
# batch.keys()

In [55]:
class VideoModel(LightningModule):
    def __init__(self, ):
        super(VideoModel, self).__init__()

        self.video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(400, 1)

        self.lr = 1e-3
        self.batch_size = 8
        self.num_worker = 4
        self.num_steps_train = 0
        self.num_steps_val = 0

        # self.metric = torchmetrics.classification.MultilabelAccuracy(num_labels=num_classes)
        self.metric = torchmetrics.Accuracy()
        
        #loss
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.fc(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr = self.lr)
        scheduler = ReduceLROnPlateau(opt, mode="min", factor=0.05, patience=2, min_lr=1e-6)
        # scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt,
                'lr_scheduler': scheduler, 
                "monitor": "val_loss"}

    def train_dataloader(self):
        dataset = labeled_video_dataset(train_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        # label = F.one_hot(label, num_classes=num_classes).float()
        out = self(video)
        print(f"Label: {label}\nPred: {out}")
        print(f">>> Training step No.{self.num_steps_train}:")
        # print("Pred:", out)
        # print("GT:", label)
        # print(f"Pred:\n{out}")
        # print(f"Pred shape:\n{out.shape}")
        # print(f"Label:\n{label}")
        # print(f"Label shape:\n{label.shape}")
        # print(">>> INFO: Computing Training Loss")
        
        loss = self.criterion(out, label.float().unsqueeze(1))
        
        print(f"Loss: {loss}")
        self.num_steps_train += 1
        # print(">>> INFO: Training Loss Computed")
        # print(">>> INFO: Computing Training Metric")
        # metric = self.metric(out, label)
        
        # Below is for Accuracy
        metric = self.metric(out, label.to(torch.int64).unsqueeze(1))
        
        print(f"Accuracy: {metric}")

        values = {"loss": loss,
                "metric": metric.detach()}
        
        self.log_dict({"step_loss": loss,
                        "step_metric": metric.detach()})
        
        return values
        
        # return {"loss": loss}

    def training_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        
        self.log('training_loss', loss)
        print(f">>> Epoch end loss: {loss}")
        self.log('training_metric', metric)
        

    def val_dataloader(self):
        dataset = labeled_video_dataset(val_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        # label = F.one_hot(label, num_classes=num_classes).float()
        out = self(video)
        print(f"Label: {label}\nPred: {out}")
        # print(">>> INFO: Computing Val Loss")
        print(f">>> Validation step No.{self.num_steps_val}:")

        loss = self.criterion(out, label.float().unsqueeze(1))
        print()
        print(f"Loss: {loss}")
        self.num_steps_val += 1
        # print(">>> INFO: Val Loss Computed")
        # print(">>> INFO: Computing Val Metric")
        # metric = self.metric(out, label)
        # Below is for Accuracy
        
        metric = self.metric(out, label.to(torch.int64).unsqueeze(1))
                
        print(f"Accuracy: {metric}")
        

        return {"loss": loss,
                "metric": metric.detach()}
        
        # return {"loss": loss}

    def validation_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('val_loss', loss)
        self.log('val_metric', metric)

    def test_dataloader(self):
        dataset = labeled_video_dataset(test_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        # label = F.one_hot(label, num_classes=num_classes).float()
        out = self.forward(video)
        # metric = self.metric(out, label)

        return {"label": label,
                "pred": out.detach(),}

    def test_epoch_end(self, outputs):
        label=torch.cat([x['label'] for x in outputs]).cpu().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()
        # self.log('test_loss', loss)
        # self.log('test_metric', metric)
        
        # Below is for MultiLabelClassification
        # class_labels = label.argmax(axis=1)
        # class_pred = pred.argmax(axis=1)
        
        
        # Below is for BinaryClassification
        class_labels = label
        class_pred = np.where(pred > 0, 1, 0)
        
        print(f">> Label: {class_labels}")
        print(f">> Pred: {class_pred.squeeze()}")
              
        print(classification_report(class_labels, class_pred))
        
        return {"prediction": class_pred,
                "labels": class_labels}




In [56]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath="checkpoints", 
                                    verbose=True, save_last=True, save_top_k=2)
lr_monitor = LearningRateMonitor(logging_interval="epoch")


# Training

In [57]:
model = VideoModel()
seed_everything(0)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main
Global seed set to 0


0

In [58]:
trainer = Trainer(max_epochs=100,
                accelerator="gpu", devices=-1,
                precision=16,
                # accumulate_grad_batches=2,
                enable_progress_bar=True,
                # num_sanity_val_steps=0,
                callbacks=[lr_monitor, checkpoint_callback],
                log_every_n_steps=5,
                limit_train_batches=25,
                limit_val_batches=10,
                limit_test_batches=10,)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [59]:
trainer.fit(model, ckpt_path="/home/toghrul/SLR/sign-lang/checkpoints/last-v4.ckpt")
# trainer.fit(model)

Restoring states from the checkpoint path at /home/toghrul/SLR/sign-lang/checkpoints/last-v4.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params
--------------------------------------------------
0 | video_model | EfficientX3d      | 3.8 M 
1 | relu        | ReLU              | 0     
2 | fc          | Linear            | 401   
3 | metric      | Accuracy          | 0     
4 | criterion   | BCEWithLogitsLoss | 0     
--------------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
7.589     Total estimated model params size (MB)
Restored all states from the checkpoint file at /home/toghrul/SLR/sign-lang/checkpoints/last-v4.ckpt


Sanity Checking: 0it [00:00, ?it/s]

Label: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Pred: tensor([[-19.0625],
        [-17.3438],
        [-20.7031],
        [-17.5781],
        [-21.2969],
        [-23.8906],
        [-21.2344],
        [-18.5312]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.0:

Loss: 0.0
Accuracy: 1.0
Label: tensor([0, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
Pred: tensor([[-11.0781],
        [ 15.1406],
        [-11.1328],
        [-19.9375],
        [-14.9062],
        [-16.7344],
        [-10.3125],
        [-21.0781]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.1:

Loss: 8.001826245163102e-06
Accuracy: 1.0


Training: 25it [00:00, ?it/s]

Label: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Pred: tensor([[-13.3594],
        [-16.2188],
        [-11.3828],
        [-24.3906],
        [-10.9531],
        [-13.7031],
        [-19.1875],
        [-14.5703]], device='cuda:0', dtype=torch.float16,
       grad_fn=<AddmmBackward0>)
>>> Training step No.0:
Loss: 4.023286237497814e-06
Accuracy: 1.0
Label: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Pred: tensor([[-10.2031],
        [-22.9375],
        [-20.4844],
        [ -8.8203],
        [-13.8438],
        [-18.5625],
        [-14.6562],
        [-17.0156]], device='cuda:0', dtype=torch.float16,
       grad_fn=<AddmmBackward0>)
>>> Training step No.1:
Loss: 2.3274165869224817e-05
Accuracy: 1.0
Label: tensor([0, 0, 0, 0, 1, 1, 0, 0], device='cuda:0')
Pred: tensor([[-18.2031],
        [-14.3281],
        [-34.1250],
        [-25.7656],
        [ 12.8203],
        [  9.5234],
        [-20.4375],
        [ -9.2109]], device='cuda:0', dtype=torch.float16,
       grad_fn=

...

Validation: 0it [00:00, ?it/s]

Label: tensor([0, 0, 0, 0, 0, 1, 0, 0], device='cuda:0')
Pred: tensor([[-17.6094],
        [-15.0859],
        [ -6.7266],
        [ -9.1641],
        [-13.6094],
        [ 25.0625],
        [-14.2891],
        [ -7.2695]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.112:

Loss: 0.00025011447723954916
Accuracy: 1.0
Label: tensor([0, 0, 0, 0, 1, 0, 0, 0], device='cuda:0')
Pred: tensor([[-11.2578],
        [ -7.2773],
        [-12.0938],
        [-21.2500],
        [ 14.8438],
        [ -9.1641],
        [-21.5938],
        [ -8.0391]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.113:

Loss: 0.00014212005771696568
Accuracy: 1.0
Label: tensor([1, 0, 0, 0, 0, 0, 1, 0], device='cuda:0')
Pred: tensor([[ 21.2656],
        [-13.3594],
        [ -4.9180],
        [-17.4531],
        [ -5.7969],
        [-10.7656],
        [ 30.5938],
        [ -9.7500]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.114:

Loss: 0.0013000830076634884
Accuracy: 1.0

Epoch 99, global step 2500: 'val_loss' was not in top 2


Label: tensor([1, 0, 0, 0, 1, 0, 1, 0], device='cuda:0')
Pred: tensor([[ 14.7422],
        [-10.7109],
        [-11.7188],
        [-14.0469],
        [ 18.1250],
        [-10.1406],
        [ 14.5625],
        [-21.7656]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.120:

Loss: 8.940563930082135e-06
Accuracy: 1.0
Label: tensor([1, 0, 0, 0, 1, 0, 0, 0], device='cuda:0')
Pred: tensor([[ 22.2812],
        [-18.1406],
        [ -9.3594],
        [-14.7500],
        [ 19.3281],
        [-19.4375],
        [-20.7031],
        [ -7.6367]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.121:

Loss: 7.110823935363442e-05
Accuracy: 1.0
>>> Epoch end loss: 0.0


`Trainer.fit` stopped: `max_epochs=100` reached.


In [20]:
# model = VideoModel.load_from_checkpoint(
#     checkpoint_path="/home/toghrul/SLR/sign-lang/checkpoints/last-v4.ckpt",
#     hparams_file="/home/toghrul/SLR/sign-lang/lightning_logs/version_45/hparams.yaml",
#     map_location=None,
# )
# model = model.cuda()

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [22]:
model_path = "../models/eff3d_bin.pt"
# torch.save(model.state_dict(), model_path)

# Testing

In [47]:
test_res = trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

>> Label: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0]
>> Pred: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        68
           1       1.00      0.92      0.96        12

    accuracy                           0.99        80
   macro avg       0.99      0.96      0.97        80
weighted avg       0.99      0.99      0.99        80



# Inferencing

In [21]:
from pytorchvideo.data.encoded_video import EncodedVideo

In [60]:
model_path = "../models/eff3d_bin.pt"
video_path = "../data/binary-data/test/MƏN/2022-04-26 12-58-07.mp4"
video = EncodedVideo.from_path(video_path)

In [61]:
video_data = video.get_clip(0, 1)
print(video_data['video'].shape)
video_data = video_transform(video_data)

inputs = video_data["video"].cuda()
inputs = inputs.unsqueeze(0)
inputs.shape

torch.Size([3, 15, 960, 1280])


torch.Size([1, 3, 25, 256, 256])

In [62]:
video_data['video'].shape

torch.Size([3, 25, 256, 256])

In [63]:
preds = model(inputs).detach().cpu().numpy()
preds = np.where(preds > 0, 1, 0)
bin_idx_to_class[preds[0, 0]]

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [34]:
# model_path = "../models/eff3d_bin.pt"
video_path = "../data/binary-data/test/VAR/2022-05-24 15-56-37.mp4"

In [66]:
def word_level_prediction(path_to_model, path_to_video):
    
    model = VideoModel()
    # model.load_state_dict(torch.load(path_to_model))
    
    model = VideoModel.load_from_checkpoint(
    checkpoint_path="/home/toghrul/SLR/sign-lang/checkpoints/epoch=32-step=825.ckpt",
    # hparams_file="/home/toghrul/SLR/sign-lang/lightning_logs/version_49/hparams.yaml",
    map_location=None,
    )
    
    model = model.cuda()
    
    video = EncodedVideo.from_path(path_to_video)

    video_data = video.get_clip(0, 2)
    video_data = video_transform(video_data)

    inputs = video_data["video"].cuda()
    inputs = inputs.unsqueeze(0)
    
    preds = model(inputs).detach().cpu().numpy()
    # preds = np.where(preds > 0, 1, 0)
    
    return preds

In [71]:
word_level_prediction(model_path, video_path)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main
Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main


array([[-6.538341]], dtype=float32)

# Validation

In [ ]:
# val_res = trainer.validate(model)